# Gaussian Process FEM Method on the 1D Dirichlet Problem

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import probnum as pn

import linpde_gp

In [ ]:
import experiment_utils
from experiment_utils import config

config.experiment_name = "0002_poisson_dirichlet_gp_fem"
config.target = "jmlr"
config.debug_mode = True

plt.rcParams.update(config.tueplots_bundle())

## Problem Definition

In [ ]:
bvp = linpde_gp.problems.pde.PoissonEquationDirichletProblem(
    pde=linpde_gp.problems.pde.PoissonEquation(
        domain=linpde_gp.domains.asdomain([-1.0, 1.0]),
        rhs=linpde_gp.functions.Constant(input_shape=(), value=2.0),
    ),
    boundary_values=np.asarray((0.0, 1.0)),
)

## Test and Trial Bases

In [ ]:
num_finite_elements = 5

basis_grid = np.linspace(*bvp.domain, num_finite_elements + 2)

trial_basis = linpde_gp.functions.bases.UnivariateLinearInterpolationBasis(
    basis_grid,
    zero_boundary=False,
)

test_basis = linpde_gp.functions.bases.UnivariateLinearInterpolationBasis(
    basis_grid,
    zero_boundary=True,
)

trial_proj = trial_basis.l2_projection()
test_proj = test_basis.l2_projection(normalized=False)

## Plotting

In [ ]:
%matplotlib inline

import matplotlib.axes

from probnum.typing import ArrayLike

from linpde_gp.typing import RandomProcessLike, RandomVariableLike


plt_grid = bvp.domain.uniform_grid(200)


def plot_belief(
    ax: matplotlib.axes.Axes,
    *,
    u: pn.randprocs.GaussianProcess,
    projection: linpde_gp.linfunctls.projections.l2.L2Projection_UnivariateLinearInterpolationBasis | None = None,
    conditioned_on: list[str] = [],
    Y_bc: ArrayLike | None = None,
    noise_bc: RandomVariableLike | None = None,
    X_u_meas: ArrayLike | None = None,
    Y_u_meas: ArrayLike | None = None,
    noise_u_meas: RandomVariableLike | None = None,
    show_projection_basis: linpde_gp.linfunctls.projections.l2.L2Projection_UnivariateLinearInterpolationBasis | None = None,
    solution: RandomProcessLike | None = None,
):
    # Solution Belief
    Pu = u
    label = "u"

    if projection is not None:
        Pu = linpde_gp.randprocs.ParametricGaussianProcess(
            weights=projection(u),
            feature_fn=projection.basis,
        )
        label = r"\mathcal{P}[u]"

    cond_events_str = _build_cond_events_str(conditioned_on)

    if len(cond_events_str) > 0:
        label += fr" \mid {cond_events_str}"

    Pu.plot(
        ax,
        plt_grid,
        num_samples=3,
        rng=np.random.default_rng(24),
        color="C0",
        label=f"${label}$",
    )

    # True Solution
    if solution is not None:
        linpde_gp.randprocs.asrandproc(solution).plot(
            ax,
            plt_grid,
            color="C1",
            label="$u^\star$",
        )

    for key in conditioned_on:
        match key:
            case "bc":
                X_bc = np.asarray(bvp.domain.boundary)
                Y_bc = pn.randvars.asrandvar(Y_bc)
                label = r"u(\partial D)"

                if noise_bc is not None:
                    Y_bc += pn.randvars.asrandvar(noise_bc)

                    label += r" + \epsilon_{\partial D}"

                ax.errorbar(
                    X_bc,
                    Y_bc.mean,
                    yerr=1.96 * Y_bc.std,
                    fmt="+",
                    capsize=2,
                    color="C5",
                    label=f"${label}$",
                )
            case "u_meas":
                X_u_meas = np.asarray(X_u_meas)
                Y_u_meas = np.asarray(Y_u_meas)
                noise_u_meas = pn.randvars.asrandvar(noise_u_meas)

                ax.errorbar(
                    X_u_meas,
                    Y_u_meas,
                    yerr=1.96 * noise_u_meas.std,
                    fmt="+",
                    capsize=2,
                    color="C4",
                    label=r"$u(X_{\mathrm{MEAS}}) + \epsilon_{\mathrm{MEAS}}$",
                )

    if show_projection_basis is not None:
        _plot_projection_basis(
            ax,
            u=u,
            projection=show_projection_basis,
            color="black",
            alpha=0.1,
        )

    ax.legend()


def _plot_projection_basis(
    ax: matplotlib.axes.Axes,
    u: pn.randprocs.GaussianProcess,
    projection: linpde_gp.linfunctls.projections.l2.L2Projection_UnivariateLinearInterpolationBasis,
    **plot_kwargs,
):
    Pu = projection(u)

    ax.plot(
        plt_grid,
        Pu.mean * projection.basis(plt_grid),
        **plot_kwargs,
    )


def _build_cond_events_str(conditioned_on: list[str]) -> str:
    events = []

    for key in conditioned_on:
        match key:
            case "bc":
                events.append(r"\mathrm{BC}")
            case "pde":
                events.append(r"\mathrm{PDE}")
            case "u_meas":
                events.append(r"\mathrm{MEAS}")
            case _:
                raise ValueError(f"Unknown event '{key}'")
    
    return ", ".join(events)

## Prior

In [ ]:
u_prior = pn.randprocs.GaussianProcess(
    mean=pn.functions.Zero(
        input_shape=(),
        output_shape=(),
    ),
    cov=pn.randprocs.kernels.Matern(
        input_shape=(),
        lengthscale=1.0,
        nu=1.5,
    ),
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_prior,
    solution=bvp.solution,
)

experiment_utils.savefig("00_u_prior")

In [ ]:
plot_belief(
    plt.gca(),
    u=u_prior,
    projection=trial_proj,
    solution=bvp.solution,
)

experiment_utils.savefig("00_u_prior_proj")

## Solving the Exact Problem

### Conditioning on the Boundary Conditions

In [ ]:
X_bc = np.asarray(bvp.domain.boundary)
Y_bc = bvp.boundary_conditions[0].values.support

u_cond_bc = u_prior.condition_on_observations(
    Y_bc,
    X=X_bc,
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc,
    conditioned_on=["bc"],
    Y_bc=Y_bc,
    solution=bvp.solution,
)

experiment_utils.savefig("01_00_u_cond_bc")

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc,
    projection=trial_proj,
    conditioned_on=["bc"],
    Y_bc=Y_bc,
    solution=bvp.solution,
)

experiment_utils.savefig("01_00_u_cond_bc_proj")

### Conditioning on the PDE

In [ ]:
diffop_galerkin = bvp.pde.diffop.weak_form(test_basis)(trial_basis)
rhs_galerkin = test_proj(bvp.pde.rhs)

In [ ]:
u_cond_bc_pde = u_cond_bc.condition_on_observations(
    np.zeros_like(rhs_galerkin.mean),
    L=diffop_galerkin @ trial_proj,
    b=-rhs_galerkin,
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc_pde,
    conditioned_on=["bc", "pde"],
    Y_bc=Y_bc,
    solution=bvp.solution,
)

experiment_utils.savefig("01_01_u_cond_bc_pde")

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc_pde,
    projection=trial_proj,
    conditioned_on=["bc", "pde"],
    Y_bc=Y_bc,
    solution=bvp.solution,
)

experiment_utils.savefig("01_01_u_cond_bc_pde_proj")

## Solving the Noisy Problem

### Synthetic Measurements of the Boundary Values and Initial Conditions

In [ ]:
meas_rng = np.random.default_rng(21489)

In [ ]:
X_bc = np.asarray(bvp.domain.boundary)

noise_bc_meas = pn.randvars.Normal(
    mean=np.zeros((2,)),
    cov=0.2 ** 2 * np.eye(2),
)

Y_bc_meas = bvp.boundary_conditions[0].values.support + noise_bc_meas.sample(meas_rng)

In [ ]:
num_rhs_meas = 5

X_rhs_meas = np.linspace(*bvp.domain, num_rhs_meas + 2)[1:-1]

noise_rhs_meas = pn.randvars.Normal(
    mean=np.zeros_like(X_rhs_meas),
    cov=0.3 ** 2 * np.eye(num_rhs_meas),
)

Y_rhs_meas = bvp.pde.rhs.as_fn()(X_rhs_meas) + noise_rhs_meas.sample(meas_rng)

In [ ]:
num_u_meas = 3

X_u_meas = np.linspace(*bvp.domain, num_u_meas + 2)[1:-1]

noise_u_meas = pn.randvars.Normal(
    mean=np.zeros_like(X_u_meas),
    cov=0.1 ** 2 * np.eye(num_u_meas),
)

Y_u_meas = bvp.solution.as_fn()(X_u_meas) + noise_u_meas.sample(meas_rng)

### RHS Belief

In [ ]:
f_prior = pn.randprocs.GaussianProcess(
    mean=linpde_gp.functions.Constant(
        input_shape=(),
        value=1.5,
    ),
    cov=pn.randprocs.kernels.Matern(
        input_shape=(),
        lengthscale=1.0,
        nu=0.5,
    )
)

In [ ]:
f_prior.plot(
    plt.gca(),
    plt_grid,
    label="f",
)

bvp.pde.rhs.plot(
    plt.gca(),
    plt_grid,
    label="$f^\star$"
)

plt.legend()
plt.show()

In [ ]:
f_cond_meas = f_prior.condition_on_observations(
    Y_rhs_meas,
    X=X_rhs_meas,
    b=noise_rhs_meas,
)

In [ ]:
f_cond_meas.plot(
    plt.gca(),
    plt_grid,
    label=r"$f \mid \text{MEAS}$",
)

bvp.pde.rhs.plot(
    plt.gca(),
    plt_grid,
    label="$f^\star$"
)

plt.legend()
plt.show()

### Conditioning on the Measured Boundary Conditions

In [ ]:
u_cond_bc = u_prior.condition_on_observations(
    Y_bc_meas,
    X=X_bc,
    b=noise_bc_meas,
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc,
    conditioned_on=["bc"],
    Y_bc=Y_bc_meas,
    noise_bc=noise_bc_meas,
    solution=bvp.solution,
)

experiment_utils.savefig("02_00_u_cond_bc")

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc,
    projection=trial_proj,
    conditioned_on=["bc"],
    Y_bc=Y_bc_meas,
    noise_bc=noise_bc_meas,
    solution=bvp.solution,
)

experiment_utils.savefig("02_00_u_cond_bc_proj")

### Conditioning on the PDE with Uncertain RHS

In [ ]:
diffop_galerkin = bvp.pde.diffop.weak_form(test_basis)(trial_basis)
rhs_galerkin = test_proj(f_cond_meas)

In [ ]:
u_cond_bc_pde = u_cond_bc.condition_on_observations(
    np.zeros_like(rhs_galerkin.mean),
    L=diffop_galerkin @ trial_proj,
    b=-rhs_galerkin,
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc_pde,
    conditioned_on=["bc", "pde"],
    Y_bc=Y_bc_meas,
    noise_bc=noise_bc_meas,
    show_projection_basis=trial_proj,
    solution=bvp.solution,
)

experiment_utils.savefig("02_01_u_cond_bc_pde")

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc_pde,
    projection=trial_proj,
    conditioned_on=["bc", "pde"],
    Y_bc=Y_bc_meas,
    noise_bc=noise_bc_meas,
    solution=bvp.solution,
)

experiment_utils.savefig("02_01_u_cond_bc_pde_proj")

### Conditioning on Measurements of the Solution

In [ ]:
u_cond_bc_pde_meas = u_cond_bc_pde.condition_on_observations(
    Y_u_meas,
    X=X_u_meas,
    b=noise_u_meas,
)

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc_pde_meas,
    conditioned_on=["bc", "pde", "u_meas"],
    Y_bc=Y_bc_meas,
    noise_bc=noise_bc_meas,
    X_u_meas=X_u_meas,
    Y_u_meas=Y_u_meas,
    noise_u_meas=noise_u_meas,
    solution=bvp.solution,
)

experiment_utils.savefig("02_02_u_cond_bc_pde_meas")

In [ ]:
plot_belief(
    plt.gca(),
    u=u_cond_bc_pde_meas,
    projection=trial_proj,
    conditioned_on=["bc", "pde", "u_meas"],
    Y_bc=Y_bc_meas,
    noise_bc=noise_bc_meas,
    X_u_meas=X_u_meas,
    Y_u_meas=Y_u_meas,
    noise_u_meas=noise_u_meas,
    solution=bvp.solution,
)

experiment_utils.savefig("02_02_u_cond_bc_pde_meas_proj")